In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""

# State and Output Schema

In [2]:
from typing import TypedDict

from langchain_core.pydantic_v1 import BaseModel, Field


# Pydantic model to structure the LLM's output
class ScorePrediction(BaseModel):
    """The structured output for the resume score."""

    score: float = Field(description="The matching score between JD and Resume (0–10)")
    explanation: str = Field(description="The explanation of the matching score")


# The state that will be passed through the graph
class GraphState(TypedDict):
    """The state of our graph."""

    jd: str
    resume: str
    prediction: ScorePrediction  # The final output will be stored here

/Users/nhannguyen/Documents/HCMUS/LLM/Code/llm-resume-evaluation/.env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Nodes

In [3]:
import re

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# The complete, optimized prompt generated by DSPy
OPTIMIZED_PROMPT_TEMPLATE = """
# SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text
Your output fields are:
1. `score` (str): The matching score between JD and Resume (0–10)
2. `explanation` (str): The explanation of the matching score
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## jd ## ]]
{{jd}}

[[ ## resume ## ]]
{{resume}}

[[ ## score ## ]]
{{score}}

[[ ## explanation ## ]]
{{explanation}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given a job description and a resume, predict a matching score (0-10).

# FEW-SHOT EXAMPLE 1
[[ ## jd ## ]]
Software Engineer needed with experience in Git, Docker, Java, REST APIs. Hit summer discussion culture measure ever.

[[ ## resume ## ]]
Experienced professional skilled in System Design, past, Java, foot. Thank case rather generation inside. Raise new structure race.

[[ ## score ## ]]
4

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 2
[[ ## jd ## ]]
ML Engineer needed with experience in Python, PyTorch, Computer Vision, TensorFlow, Keras. Could chair beautiful social both few through. Entire card much rate politics their identify. Pass sing goal during be those.

[[ ## resume ## ]]
Experienced professional skilled in Computer Vision, Python, MLOps, PyTorch, TensorFlow, parent. Exactly section network detail. Short out team author deal hospital able.

[[ ## score ## ]]
10

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 3
[[ ## jd ## ]]
ML Engineer needed with experience in TensorFlow, Computer Vision, Keras, PyTorch, MLOps. Approach wish fine near. Agree long behind stuff how positive tree. Quality team general office painting official.

[[ ## resume ## ]]
Experienced professional skilled in Cloud, interest, MLOps, Keras, hotel, PyTorch. Address guy fund window well impact quite. Place raise really feeling vote per. Catch board present market society fight foreign. Out generation beyond six degree stop.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 4
[[ ## jd ## ]]
Data Scientist needed with experience in Deep Learning, Machine Learning, NLP, SQL, Pandas, Python, Statistics. Nearly computer close garden. Law individual business hair show. Final though kitchen purpose five. Use sea right civil.

[[ ## resume ## ]]
Experienced professional skilled in Python, Deep Learning, garden, Statistics, SQL, Pandas, stock. Note operation despite born. Step take share million message long board.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FINAL QUERY
[[ ## jd ## ]]
{jd}

[[ ## resume ## ]]
{resume}

Respond with the corresponding output fields, starting with the field `[[ ## score ## ]]`, then `[[ ## explanation ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.
"""


def scorer_node(state: GraphState):
    """Invokes the LLM to score the resume against the job description."""

    model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    # Create the prompt from the template
    prompt = ChatPromptTemplate.from_template(OPTIMIZED_PROMPT_TEMPLATE)

    # Create the chain
    chain = prompt | model

    # Get inputs from the state
    jd = state["jd"]
    resume = state["resume"]

    # Invoke the chain
    response = chain.invoke({"jd": jd, "resume": resume})
    response_text = response.content

    # Parse the structured output from the model's response
    try:
        score_match = re.search(r"\[\[ ## score ## \]\]\s*([\d.]+)", response_text)
        explanation_match = re.search(
            r"\[\[ ## explanation ## \]\]\s*([\s\S]*?)\s*\[\[ ## completed ## \]\]",
            response_text,
        )

        if not score_match or not explanation_match:
            raise ValueError("Could not find score or explanation tags in the output.")

        score_val = float(score_match.group(1).strip())
        explanation_val = explanation_match.group(1).strip()

        prediction = ScorePrediction(score=score_val, explanation=explanation_val)

    except (ValueError, AttributeError) as e:
        print(f"Error parsing LLM output: {e}\nResponse: {response_text}")
        # Assign a default low score on parsing failure
        prediction = ScorePrediction(
            score=0.0, explanation="Failed to parse model output."
        )

    return {"prediction": prediction}

/Users/nhannguyen/Documents/HCMUS/LLM/Code/llm-resume-evaluation/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Graph

In [4]:
from langgraph.graph import END, StateGraph

# Initialize the graph
workflow = StateGraph(GraphState)

# Add the scorer node
workflow.add_node("scorer", scorer_node)

# Set the entry point and the final edge
workflow.set_entry_point("scorer")
workflow.add_edge("scorer", END)

# Compile the graph into a runnable application
app = workflow.compile()

print("\n✅ LangGraph App Compiled Successfully!")

# --- Example Usage ---
jd_example = "Product Manager needed with experience in User Stories, Product Roadmap, Stakeholder Management, Agile."
resume_example = "Experienced professional skilled in blood, Scrum, skill, Stakeholder Management, quality, Agile."

inputs = {"jd": jd_example, "resume": resume_example}

# Run the graph
result = app.invoke(inputs)

print("\n--- Example Result ---")
print(f"Score: {result['prediction'].score}")
print(f"Explanation: {result['prediction'].explanation}")

E0000 00:00:1760460230.526915 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



✅ LangGraph App Compiled Successfully!

--- Example Result ---
Score: 7.0
Explanation: The resume shows a good match with "Stakeholder Management" and "Agile" which are directly mentioned in the job description. Additionally, "Scrum" is closely related to Agile methodologies and is a relevant skill for a Product Manager. However, "User Stories" and "Product Roadmap," which are key responsibilities for a Product Manager, are not explicitly mentioned in the resume.


# Metrics

In [5]:
import numpy as np
from sklearn.metrics import mean_absolute_error


def accuracy_at_threshold(gold_scores, pred_scores, threshold):
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)
    absolute_errors = np.abs(gold_scores - pred_scores)
    within_threshold_count = np.sum(absolute_errors <= threshold)
    return within_threshold_count / len(gold_scores)


# Evaluate

In [6]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
val_df = pd.read_csv("../../data/test.csv")

## Kaggle dataset

In [7]:
import time

from tqdm import tqdm

gold_scores = []
pred_scores = []

print(f"\n--- Starting Evaluation on {len(val_df)} examples ---")

for i, example in tqdm(
    val_df.iterrows(), desc="Evaluating LangGraph", total=len(val_df)
):
    # Get ground truth data
    jd = example["job_description"]
    resume = example["resume"]
    gold_score = example["match_score"]

    # Run the LangGraph app
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)

print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")


--- Starting Evaluation on 3000 examples ---


Evaluating LangGraph:   0%|          | 0/3000 [00:00<?, ?it/s]

E0000 00:00:1760460237.031020 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Evaluating LangGraph:   0%|          | 8/3000 [01:09<7:14:19,  8.71s/it]E0000 00:00:1760460306.801994 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
9

[[ ## explanation ## ]]
The resume shows strong alignment with the job description, matching key skills such as Git, Java, System Design, and Docker. The only skill explicitly mentioned in the JD but not directly in the resume is "REST APIs," although "Spring Boot" in the resume often implies experience with building REST APIs in Java.


Evaluating LangGraph:   1%|          | 17/3000 [02:04<4:35:13,  5.54s/it]E0000 00:00:1760460361.676130 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


...

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
8

[[ ## explanation ## ]]
The resume shows strong alignment with the job description, matching 3 out of 4 key skills: Keras, MLOps, and TensorFlow. The resume also includes relevant skills like Python and Computer Vision, which are beneficial for an ML Engineer role. The primary missing skill from the resume is PyTorch.


Evaluating LangGraph:  99%|█████████▉| 2967/3000 [4:55:40<02:38,  4.81s/it]E0000 00:00:1760477977.871905 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
8

[[ ## explanation ## ]]
The resume shows strong alignment with the job description, matching key skills such as Docker, Spring Boot, REST APIs, and Git. However, the resume does not explicitly mention Java, which is a required skill in the job description.


Evaluating LangGraph: 100%|██████████| 3000/3000 [4:59:06<00:00,  5.98s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 1.1093
---
🎯 Accuracy@1 (error <= 1.0): 79.10%
🎯 Accuracy@2 (error <= 2.0): 93.07%
🎯 Accuracy@3 (error <= 3.0): 94.23%
--------------------------------------


In [8]:
import json

with open("preds/preds_gemini_kaggle.json", "w") as f:
    json.dump(pred_scores, f)

## Collected dataset

In [9]:
import json
import time

from tqdm import tqdm

collected_folder = "/Users/nhannguyen/ngtuthanhan@gmail.com - Google Drive/My Drive/HCMUS/LLM/Data_Collector_30_8"
matching_json = os.path.join(collected_folder, "Matching_Result.json")

with open(matching_json, "r") as f:
    matching_data = json.load(f)

pred_scores = []
gold_scores = []

for example in tqdm(
    matching_data, desc="Evaluating LangGraph", total=len(matching_data)
):
    jd_path = example["JD"]
    resume_path = example["CV"]
    gold_score = example["Score"]
    with open(os.path.join(collected_folder, "JD", jd_path) + ".txt", "r") as f:
        jd = f.read()
    with open(os.path.join(collected_folder, "CV", resume_path) + ".txt", "r") as f:
        resume = f.read()
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)


print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")

Evaluating LangGraph:   0%|          | 0/87 [00:00<?, ?it/s]E0000 00:00:1760478185.233015 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
5

[[ ## explanation ## ]]
The candidate demonstrates a strong foundational understanding of Machine Learning, Deep Learning, Computer Vision, and NLP through their education, research assistant role, and projects (e.g., Face-mask Detection, Vietnamese Scene Text Recognition). They also have relevant skills like Python, TensorFlow, PyTorch, Pandas, Git, and Docker, and have achieved success in ML competitions, which is a plus point for the JD.

However, there are significant gaps. The job description explicitly asks for "2-4 years of experience in building production models as a Data Scientist/ML Engineer," which the candidate, being a current student (2019-Present education, 2021-Present experience), does not meet. The resume also lacks explicit experience with key technologies mentioned in the JD for big data and production, such as Scala, SQL (beyond general database design 

Evaluating LangGraph:   1%|          | 1/87 [00:19<28:07, 19.63s/it]E0000 00:00:1760478204.335245 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
8

[[ ## explanation ## ]]
The candidate demonstrates a strong match with the core technical requirements of the AI Engineer role. They have proven experience in developing and fine-tuning machine learning models for computer vision and natural language processing through their projects (Face-mask Detection, Vietnamese Scene Text Recognition) and AI Club activities. Proficiency in Python, TensorFlow, and PyTorch is explicitly listed and supported by project work. The candidate's role as a Research Assistant and participation in the Ho Chi Minh AI Challenge align well with the JD's emphasis on research and experimentation.

However, the candidate is currently a student (2019 - Present), which might be a slight mismatch for the "proven experience" requirement typically implying a graduate. While they show some integration experience with web applications and frameworks, explicit 

Evaluating LangGraph:   2%|▏         | 2/87 [00:35<24:26, 17.25s/it]E0000 00:00:1760478219.857697 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


...

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
4

[[ ## explanation ## ]]
The candidate demonstrates a good foundation in the required core technologies, including Java (Spring Boot), TypeScript (ReactJS), PostgreSQL, and Docker, which aligns well with the job's technical stack for a Full Stack Engineer. They also have relevant full-stack project experience and an internship.

However, there are significant mismatches:
1.  **Experience Level:** The job requires 3-5 years of practical experience, while the candidate is a fresher with only about 7 months of professional/internship experience. This is the primary reason for the low score.
2.  **English Communication:** The JD explicitly asks for "Good at English communication," but the resume states "English: Basic communication."
3.  **Cloud Platforms:** The JD lists AWS/GCP under Infra, which is not explicitly mentioned in the candidate's skills or experience.
4.  **Specific

Evaluating LangGraph:  99%|█████████▉| 86/87 [20:06<00:13, 13.05s/it]E0000 00:00:1760479391.510757 25850731 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
3

[[ ## explanation ## ]]
The candidate demonstrates strong technical skills in Java, JavaScript, Kotlin, React, SpringBoot, and deployment practices (nginx, systemd), which align well with the Full Stack Engineer role and the specified programming languages. Their project experience showcases practical application of these skills in building and deploying web applications. However, there are significant mismatches: the job requires 3-5 years of practical experience and a completed Bachelor's degree, whereas the candidate is currently an intern and is expected to graduate in 2026. The resume also lacks explicit mention of experience with AWS/GCP infrastructure and detailed information on computer science fundamentals, data structures, algorithms, dependency injection, and comprehensive testing practices as required by the JD. The experience level is the primary reason for the 

Evaluating LangGraph: 100%|██████████| 87/87 [20:20<00:00, 14.03s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 5.7701
---
🎯 Accuracy@1 (error <= 1.0): 12.64%
🎯 Accuracy@2 (error <= 2.0): 13.79%
🎯 Accuracy@3 (error <= 3.0): 22.99%
--------------------------------------


In [10]:
import json

with open("preds/preds_gemini_collected.json", "w") as f:
    json.dump(pred_scores, f)